<a href="https://colab.research.google.com/github/areejatariq/Data-cleaning-and-visualization-using-pyspark-/blob/main/Assignment1_DSTT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#installing the pyspark

!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 39 kB/s 
     |████████████████████████████████| 199 kB 51.9 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=ec66216aeb474324a1fa9908b4c39dfede1b22c9bfec7071dd9d45dba91d90ae
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


In [2]:

#creating the pyspark session 
from pyspark.sql import SparkSession
spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

In [3]:
#checking if the pyspark session is working
spark


In [4]:
# Reading the data from the directory
df_bristol = spark.read.csv('/content/bristol-air-quality-data.csv', header=True,sep=";")


In [5]:
df_bristol.show(10)

+--------------------+--------+--------+--------+------+----+------+-----+-------+------+------+----+--------+----+-----------+----+------------+--------------------+--------------------+--------------------+--------------------+-------+--------------------+
|           Date Time|     NOx|     NO2|      NO|SiteID|PM10|NVPM10|VPM10|NVPM2.5| PM2.5|VPM2.5|  CO|      O3| SO2|Temperature|  RH|Air Pressure|            Location|        geo_point_2d|           DateStart|             DateEnd|Current|     Instrument Type|
+--------------------+--------+--------+--------+------+----+------+-----+-------+------+------+----+--------+----+-----------+----+------------+--------------------+--------------------+--------------------+--------------------+-------+--------------------+
|2013-08-23T07:00:...|51.54044|30.50055|13.72186|   452|27.8|  23.2|  4.6|   16.4|19.454|   2.9|null|20.40603|null|       null|null|        null|       AURN St Pauls|51.4628294172,-2....|2006-06-15T00:00:...|               

In [6]:
#taking the mean of the SO2 and than adding that average back into the column

from pyspark.sql.functions import mean
mean_val=df_bristol.select(mean(df_bristol.SO2)).collect()
print(type(mean_val)) #mean_val is a list row object
print('mean value of SO2', mean_val[0][0])
mean_SO2=mean_val[0][0]

#now using mean_SO2 value to fill the nulls in SO2 column
df_bristol=df_bristol.na.fill(value='7.604697174',subset=['SO2'])

<class 'list'>
mean value of SO2 7.604697174396046


In [7]:
df_bristol.show()

+--------------------+--------+--------+--------+------+----+------+-----+-------+------+------+----+--------+-----------+-----------+----+------------+--------------------+--------------------+--------------------+--------------------+-------+--------------------+
|           Date Time|     NOx|     NO2|      NO|SiteID|PM10|NVPM10|VPM10|NVPM2.5| PM2.5|VPM2.5|  CO|      O3|        SO2|Temperature|  RH|Air Pressure|            Location|        geo_point_2d|           DateStart|             DateEnd|Current|     Instrument Type|
+--------------------+--------+--------+--------+------+----+------+-----+-------+------+------+----+--------+-----------+-----------+----+------------+--------------------+--------------------+--------------------+--------------------+-------+--------------------+
|2013-08-23T07:00:...|51.54044|30.50055|13.72186|   452|27.8|  23.2|  4.6|   16.4|19.454|   2.9|null|20.40603|7.604697174|       null|null|        null|       AURN St Pauls|51.4628294172,-2....|2006-06-

In [8]:
#taking the mean of the temperature and than adding that average back into the column

from pyspark.sql.functions import mean
mean_val_temp=df_bristol.select(mean(df_bristol.Temperature)).collect()
print(type(mean_val_temp)) #mean_val_temp is a list row object
print('mean value of Temperature', mean_val_temp[0][0])
mean_temp=mean_val_temp[0][0]

#now using mean_temp value to fill the nulls in temperature column
df_bristol=df_bristol.na.fill(value="9.5137689",subset=['Temperature'])

<class 'list'>
mean value of Temperature 9.513768979472026


In [9]:
df_bristol.show()

+--------------------+--------+--------+--------+------+----+------+-----+-------+------+------+----+--------+-----------+-----------+----+------------+--------------------+--------------------+--------------------+--------------------+-------+--------------------+
|           Date Time|     NOx|     NO2|      NO|SiteID|PM10|NVPM10|VPM10|NVPM2.5| PM2.5|VPM2.5|  CO|      O3|        SO2|Temperature|  RH|Air Pressure|            Location|        geo_point_2d|           DateStart|             DateEnd|Current|     Instrument Type|
+--------------------+--------+--------+--------+------+----+------+-----+-------+------+------+----+--------+-----------+-----------+----+------------+--------------------+--------------------+--------------------+--------------------+-------+--------------------+
|2013-08-23T07:00:...|51.54044|30.50055|13.72186|   452|27.8|  23.2|  4.6|   16.4|19.454|   2.9|null|20.40603|7.604697174|  9.5137689|null|        null|       AURN St Pauls|51.4628294172,-2....|2006-06-

In [10]:
#taking the mean of the PM2.5 and than adding that average back into the column

from pyspark.sql.functions import mean

# have to rename the column pm2.5 to pm2_5 because has a . in it 

df_bristol = df_bristol.withColumnRenamed('PM2.5', 'PM2_5')

mean_val_pm=df_bristol.select(mean(df_bristol.PM2_5)).collect()
print(type(mean_val_pm)) #mean_val_pm is a list row object
print('mean value of PM2_5', mean_val_pm[0][0])
mean_PM_2=mean_val_pm[0][0]

#now using mean_temp value to fill the nulls in PM2_5 column
df_bristol=df_bristol.na.fill(value="12.052908769",subset=['PM2_5'])
df_bristol.show()                  

<class 'list'>
mean value of PM2_5 12.052908769052758
+--------------------+--------+--------+--------+------+----+------+-----+-------+------------+------+----+--------+-----------+-----------+----+------------+--------------------+--------------------+--------------------+--------------------+-------+--------------------+
|           Date Time|     NOx|     NO2|      NO|SiteID|PM10|NVPM10|VPM10|NVPM2.5|       PM2_5|VPM2.5|  CO|      O3|        SO2|Temperature|  RH|Air Pressure|            Location|        geo_point_2d|           DateStart|             DateEnd|Current|     Instrument Type|
+--------------------+--------+--------+--------+------+----+------+-----+-------+------------+------+----+--------+-----------+-----------+----+------------+--------------------+--------------------+--------------------+--------------------+-------+--------------------+
|2013-08-23T07:00:...|51.54044|30.50055|13.72186|   452|27.8|  23.2|  4.6|   16.4|      19.454|   2.9|null|20.40603|7.604697174|  

In [11]:
#taking the mean of the NVPM2.5 and than adding that average back into the column

from pyspark.sql.functions import mean

# have to rename the column NVpm2.5 to NVpm2_5 because has a . in it 

df_bristol = df_bristol.withColumnRenamed('NVPM2.5', 'NVPM2_5')

mean_val_nvpm=df_bristol.select(mean(df_bristol.NVPM2_5)).collect()
print(type(mean_val_nvpm)) #mean_val_pm is a list row object
print('mean value of NVPM2_5', mean_val_nvpm[0][0])
mean_NVPM_2=mean_val_nvpm[0][0]

#now using mean_temp value to fill the nulls in PM2_5 column
df_bristol=df_bristol.na.fill(value="9.27125962",subset=['NVPM2_5'])
df_bristol.show()                                  

<class 'list'>
mean value of NVPM2_5 9.271259623176672
+--------------------+--------+--------+--------+------+----+------+-----+----------+------------+------+----+--------+-----------+-----------+----+------------+--------------------+--------------------+--------------------+--------------------+-------+--------------------+
|           Date Time|     NOx|     NO2|      NO|SiteID|PM10|NVPM10|VPM10|   NVPM2_5|       PM2_5|VPM2.5|  CO|      O3|        SO2|Temperature|  RH|Air Pressure|            Location|        geo_point_2d|           DateStart|             DateEnd|Current|     Instrument Type|
+--------------------+--------+--------+--------+------+----+------+-----+----------+------------+------+----+--------+-----------+-----------+----+------------+--------------------+--------------------+--------------------+--------------------+-------+--------------------+
|2013-08-23T07:00:...|51.54044|30.50055|13.72186|   452|27.8|  23.2|  4.6|      16.4|      19.454|   2.9|null|20.40603|7

In [12]:
#taking the mean of the VPM2.5 and than adding that average back into the column

from pyspark.sql.functions import mean

# have to rename the column NVpm2.5 to NVpm2_5 because has a . in it 

df_bristol = df_bristol.withColumnRenamed('VPM2.5', 'VPM2_5')
   
mean_val_vpm=df_bristol.select(mean(df_bristol.VPM2_5)).collect()
print(type(mean_val_vpm)) #mean_val_vpm is a list row object
print('mean value of VPM2_5', mean_val_vpm[0][0])
mean_VPM_2=mean_val_vpm[0][0]
                 
#now using mean_temp value to fill the nulls in PM2_5 column
df_bristol=df_bristol.na.fill(value="3.2491655",subset=['VPM2_5'])
df_bristol.show()    

<class 'list'>
mean value of VPM2_5 3.2491655692868626
+--------------------+--------+--------+--------+------+----+------+-----+----------+------------+---------+----+--------+-----------+-----------+----+------------+--------------------+--------------------+--------------------+--------------------+-------+--------------------+
|           Date Time|     NOx|     NO2|      NO|SiteID|PM10|NVPM10|VPM10|   NVPM2_5|       PM2_5|   VPM2_5|  CO|      O3|        SO2|Temperature|  RH|Air Pressure|            Location|        geo_point_2d|           DateStart|             DateEnd|Current|     Instrument Type|
+--------------------+--------+--------+--------+------+----+------+-----+----------+------------+---------+----+--------+-----------+-----------+----+------------+--------------------+--------------------+--------------------+--------------------+-------+--------------------+
|2013-08-23T07:00:...|51.54044|30.50055|13.72186|   452|27.8|  23.2|  4.6|      16.4|      19.454|      2.9|nul

In [13]:
#taking the mean of the 03 and than adding that average back into the column

from pyspark.sql.functions import mean

   
mean_val_O3=df_bristol.select(mean(df_bristol.O3)).collect()
print(type(mean_val_O3)) #mean_val_O3 is a list row object
print('mean value of O3', mean_val_O3[0][0])
mean_val_O3=mean_val_O3[0][0]
                 
 #now using mean_temp value to fill the nulls in PM2_5 column
df_bristol=df_bristol.na.fill(value="40.65212",subset=['O3'])
df_bristol.show()   

<class 'list'>
mean value of O3 40.652129972769764
+--------------------+--------+--------+--------+------+----+------+-----+----------+------------+---------+----+--------+-----------+-----------+----+------------+--------------------+--------------------+--------------------+--------------------+-------+--------------------+
|           Date Time|     NOx|     NO2|      NO|SiteID|PM10|NVPM10|VPM10|   NVPM2_5|       PM2_5|   VPM2_5|  CO|      O3|        SO2|Temperature|  RH|Air Pressure|            Location|        geo_point_2d|           DateStart|             DateEnd|Current|     Instrument Type|
+--------------------+--------+--------+--------+------+----+------+-----+----------+------------+---------+----+--------+-----------+-----------+----+------------+--------------------+--------------------+--------------------+--------------------+-------+--------------------+
|2013-08-23T07:00:...|51.54044|30.50055|13.72186|   452|27.8|  23.2|  4.6|      16.4|      19.454|      2.9|null|20

In [14]:
#taking the mean of the NVPM10 and than adding that average back into the column

from pyspark.sql.functions import mean

   
mean_val_NVPM10=df_bristol.select(mean(df_bristol.NVPM10)).collect()
print(type(mean_val_NVPM10)) #mean_val_NVPM10 is a list row object
print('mean value of NVPM10', mean_val_NVPM10[0][0])
mean_val_NVPM10=mean_val_NVPM10[0][0]
                 
 #now using mean_temp value to fill the nulls in PM2_5 column
df_bristol=df_bristol.na.fill(value="14.62690601",subset=['NVPM10'])
df_bristol.show()

<class 'list'>
mean value of NVPM10 14.62690601811944
+--------------------+--------+--------+--------+------+----+-----------+-----+----------+------------+---------+----+--------+-----------+-----------+----+------------+--------------------+--------------------+--------------------+--------------------+-------+--------------------+
|           Date Time|     NOx|     NO2|      NO|SiteID|PM10|     NVPM10|VPM10|   NVPM2_5|       PM2_5|   VPM2_5|  CO|      O3|        SO2|Temperature|  RH|Air Pressure|            Location|        geo_point_2d|           DateStart|             DateEnd|Current|     Instrument Type|
+--------------------+--------+--------+--------+------+----+-----------+-----+----------+------------+---------+----+--------+-----------+-----------+----+------------+--------------------+--------------------+--------------------+--------------------+-------+--------------------+
|2013-08-23T07:00:...|51.54044|30.50055|13.72186|   452|27.8|       23.2|  4.6|      16.4|      1

In [15]:
from pyspark.sql.functions import mean

   
mean_val_PM10=df_bristol.select(mean(df_bristol.PM10)).collect()
print(type(mean_val_PM10)) #mean_val_PM10 is a list row object
print('mean value of PM10', mean_val_PM10[0][0])
mean_val_PM10=mean_val_PM10[0][0]
                 
 #now using mean_temp value to fill the nulls in PM2_5 column
df_bristol=df_bristol.na.fill(value="22.2230",subset=['PM10'])
df_bristol.show()

<class 'list'>
mean value of PM10 22.22305344257434
+--------------------+--------+--------+--------+------+-------+-----------+-----+----------+------------+---------+----+--------+-----------+-----------+----+------------+--------------------+--------------------+--------------------+--------------------+-------+--------------------+
|           Date Time|     NOx|     NO2|      NO|SiteID|   PM10|     NVPM10|VPM10|   NVPM2_5|       PM2_5|   VPM2_5|  CO|      O3|        SO2|Temperature|  RH|Air Pressure|            Location|        geo_point_2d|           DateStart|             DateEnd|Current|     Instrument Type|
+--------------------+--------+--------+--------+------+-------+-----------+-----+----------+------------+---------+----+--------+-----------+-----------+----+------------+--------------------+--------------------+--------------------+--------------------+-------+--------------------+
|2013-08-23T07:00:...|51.54044|30.50055|13.72186|   452|   27.8|       23.2|  4.6|      16

In [16]:
from pyspark.sql.functions import mean

   
mean_val_VPM10=df_bristol.select(mean(df_bristol.VPM10)).collect()
print(type(mean_val_VPM10)) #mean_val_PM10 is a list row object
print('mean value of VPM10', mean_val_VPM10[0][0])
mean_val_VPM10=mean_val_VPM10[0][0]
                 
 #now using mean_temp value to fill the nulls in PM2_5 column
df_bristol=df_bristol.na.fill(value="3.1834",subset=['VPM10'])
df_bristol.show()

<class 'list'>
mean value of VPM10 3.183470501056342
+--------------------+--------+--------+--------+------+-------+-----------+------+----------+------------+---------+----+--------+-----------+-----------+----+------------+--------------------+--------------------+--------------------+--------------------+-------+--------------------+
|           Date Time|     NOx|     NO2|      NO|SiteID|   PM10|     NVPM10| VPM10|   NVPM2_5|       PM2_5|   VPM2_5|  CO|      O3|        SO2|Temperature|  RH|Air Pressure|            Location|        geo_point_2d|           DateStart|             DateEnd|Current|     Instrument Type|
+--------------------+--------+--------+--------+------+-------+-----------+------+----------+------------+---------+----+--------+-----------+-----------+----+------------+--------------------+--------------------+--------------------+--------------------+-------+--------------------+
|2013-08-23T07:00:...|51.54044|30.50055|13.72186|   452|   27.8|       23.2|   4.6|   

In [17]:
from pyspark.sql.functions import mean

   
mean_val_CO=df_bristol.select(mean(df_bristol.CO)).collect()
print(type(mean_val_CO)) #mean_val_CO is a list row object
print('mean value of CO', mean_val_CO[0][0])
mean_val_CO=mean_val_CO[0][0]
                 
 #now using mean_temp value to fill the nulls in PM2_5 column
df_bristol=df_bristol.na.fill(value=" 0.507",subset=['CO'])
df_bristol.show()

<class 'list'>
mean value of CO 0.5077614964526624
+--------------------+--------+--------+--------+------+-------+-----------+------+----------+------------+---------+------+--------+-----------+-----------+----+------------+--------------------+--------------------+--------------------+--------------------+-------+--------------------+
|           Date Time|     NOx|     NO2|      NO|SiteID|   PM10|     NVPM10| VPM10|   NVPM2_5|       PM2_5|   VPM2_5|    CO|      O3|        SO2|Temperature|  RH|Air Pressure|            Location|        geo_point_2d|           DateStart|             DateEnd|Current|     Instrument Type|
+--------------------+--------+--------+--------+------+-------+-----------+------+----------+------------+---------+------+--------+-----------+-----------+----+------------+--------------------+--------------------+--------------------+--------------------+-------+--------------------+
|2013-08-23T07:00:...|51.54044|30.50055|13.72186|   452|   27.8|       23.2|   4.6

In [18]:
mean_val_RH=df_bristol.select(mean(df_bristol.RH)).collect()
print(type(mean_val_RH)) #mean_val_RH is a list row object
print('mean value of RH', mean_val_RH[0][0])
mean_val_RH=mean_val_RH[0][0]
                 
 #now using mean_RH value to fill the nulls in RH column
df_bristol=df_bristol.na.fill(value="25.461",subset=['RH'])
df_bristol.show()

<class 'list'>
mean value of RH 25.323765867070538
+--------------------+--------+--------+--------+------+-------+-----------+------+----------+------------+---------+------+--------+-----------+-----------+------+------------+--------------------+--------------------+--------------------+--------------------+-------+--------------------+
|           Date Time|     NOx|     NO2|      NO|SiteID|   PM10|     NVPM10| VPM10|   NVPM2_5|       PM2_5|   VPM2_5|    CO|      O3|        SO2|Temperature|    RH|Air Pressure|            Location|        geo_point_2d|           DateStart|             DateEnd|Current|     Instrument Type|
+--------------------+--------+--------+--------+------+-------+-----------+------+----------+------------+---------+------+--------+-----------+-----------+------+------------+--------------------+--------------------+--------------------+--------------------+-------+--------------------+
|2013-08-23T07:00:...|51.54044|30.50055|13.72186|   452|   27.8|       23.2|

In [19]:
mean_val_NOx=df_bristol.select(mean(df_bristol.NOx)).collect()
print(type(mean_val_NOx)) #mean_val_RH is a list row object
print('mean value of NOX', mean_val_NOx[0][0])
mean_val_NOx=mean_val_NOx[0][0]
                 
 #now using mean_RH value to fill the nulls in RH column
df_bristol=df_bristol.na.fill(value="116.892",subset=['NOx'])
df_bristol.show()

<class 'list'>
mean value of NOX 116.89264586060465
+--------------------+--------+--------+--------+------+-------+-----------+------+----------+------------+---------+------+--------+-----------+-----------+------+------------+--------------------+--------------------+--------------------+--------------------+-------+--------------------+
|           Date Time|     NOx|     NO2|      NO|SiteID|   PM10|     NVPM10| VPM10|   NVPM2_5|       PM2_5|   VPM2_5|    CO|      O3|        SO2|Temperature|    RH|Air Pressure|            Location|        geo_point_2d|           DateStart|             DateEnd|Current|     Instrument Type|
+--------------------+--------+--------+--------+------+-------+-----------+------+----------+------------+---------+------+--------+-----------+-----------+------+------------+--------------------+--------------------+--------------------+--------------------+-------+--------------------+
|2013-08-23T07:00:...|51.54044|30.50055|13.72186|   452|   27.8|       23.2

In [20]:
mean_val_NO2=df_bristol.select(mean(df_bristol.NO2)).collect()
print(type(mean_val_NO2)) #mean_val_RH is a list row object
print('mean value of NO2', mean_val_NO2[0][0])
mean_val_NO2=mean_val_NO2[0][0]
                
 #now using mean_RH value to fill the nulls in RH column
df_bristol=df_bristol.na.fill(value="43.8847",subset=['NO2'])
df_bristol.show()

<class 'list'>
mean value of NO2 43.884717236857874
+--------------------+--------+--------+--------+------+-------+-----------+------+----------+------------+---------+------+--------+-----------+-----------+------+------------+--------------------+--------------------+--------------------+--------------------+-------+--------------------+
|           Date Time|     NOx|     NO2|      NO|SiteID|   PM10|     NVPM10| VPM10|   NVPM2_5|       PM2_5|   VPM2_5|    CO|      O3|        SO2|Temperature|    RH|Air Pressure|            Location|        geo_point_2d|           DateStart|             DateEnd|Current|     Instrument Type|
+--------------------+--------+--------+--------+------+-------+-----------+------+----------+------------+---------+------+--------+-----------+-----------+------+------------+--------------------+--------------------+--------------------+--------------------+-------+--------------------+
|2013-08-23T07:00:...|51.54044|30.50055|13.72186|   452|   27.8|       23.2

In [21]:
mean_val_NO=df_bristol.select(mean(df_bristol.NO)).collect()
print(type(mean_val_NO)) #mean_val_RH is a list row object
print('mean value of NO', mean_val_NO[0][0])
mean_val_NO=mean_val_NO[0][0]
                
 #now using mean_NO value to fill the mean in NO column
df_bristol=df_bristol.na.fill(value="48.1410",subset=['NO'])
df_bristol.show()

<class 'list'>
mean value of NO 48.141030144393696
+--------------------+--------+--------+--------+------+-------+-----------+------+----------+------------+---------+------+--------+-----------+-----------+------+------------+--------------------+--------------------+--------------------+--------------------+-------+--------------------+
|           Date Time|     NOx|     NO2|      NO|SiteID|   PM10|     NVPM10| VPM10|   NVPM2_5|       PM2_5|   VPM2_5|    CO|      O3|        SO2|Temperature|    RH|Air Pressure|            Location|        geo_point_2d|           DateStart|             DateEnd|Current|     Instrument Type|
+--------------------+--------+--------+--------+------+-------+-----------+------+----------+------------+---------+------+--------+-----------+-----------+------+------------+--------------------+--------------------+--------------------+--------------------+-------+--------------------+
|2013-08-23T07:00:...|51.54044|30.50055|13.72186|   452|   27.8|       23.2|

In [22]:
# have to rename the column NVpm2.5 to NVpm2_5 because has a . in it 

df_bristol = df_bristol.withColumnRenamed('Air Pressure', 'Air_Pressure')

df_bristol=df_bristol.na.fill(value="-999",subset=['Air_Pressure'])
df_bristol.show()


+--------------------+--------+--------+--------+------+-------+-----------+------+----------+------------+---------+------+--------+-----------+-----------+------+------------+--------------------+--------------------+--------------------+--------------------+-------+--------------------+
|           Date Time|     NOx|     NO2|      NO|SiteID|   PM10|     NVPM10| VPM10|   NVPM2_5|       PM2_5|   VPM2_5|    CO|      O3|        SO2|Temperature|    RH|Air_Pressure|            Location|        geo_point_2d|           DateStart|             DateEnd|Current|     Instrument Type|
+--------------------+--------+--------+--------+------+-------+-----------+------+----------+------------+---------+------+--------+-----------+-----------+------+------------+--------------------+--------------------+--------------------+--------------------+-------+--------------------+
|2013-08-23T07:00:...|51.54044|30.50055|13.72186|   452|   27.8|       23.2|   4.6|      16.4|      19.454|      2.9| 0.507|20.

In [23]:


# have to rename the column Instrument Type to Instrument_Type because has a space in it 

df_bristol = df_bristol.withColumnRenamed('Instrument Type', 'Instrument_Type')

#to impute the categorical values we used the aggregate function in which the most occuring value was used to impute the missing values

import pyspark.sql.functions as F
for col_name in ['Current', 'Location','Instrument_Type']:
    common = df_bristol.dropna().groupBy(col_name).agg(F.count("*")).orderBy('count(1)', ascending=False).first()[col_name]
    df_bristol = df_bristol.withColumn(col_name, F.when(F.isnull(col_name), common).otherwise(df_bristol[col_name]))


df_bristol.show()

+--------------------+--------+--------+--------+------+-------+-----------+------+----------+------------+---------+------+--------+-----------+-----------+------+------------+--------------------+--------------------+--------------------+--------------------+-------+--------------------+
|           Date Time|     NOx|     NO2|      NO|SiteID|   PM10|     NVPM10| VPM10|   NVPM2_5|       PM2_5|   VPM2_5|    CO|      O3|        SO2|Temperature|    RH|Air_Pressure|            Location|        geo_point_2d|           DateStart|             DateEnd|Current|     Instrument_Type|
+--------------------+--------+--------+--------+------+-------+-----------+------+----------+------------+---------+------+--------+-----------+-----------+------+------------+--------------------+--------------------+--------------------+--------------------+-------+--------------------+
|2013-08-23T07:00:...|51.54044|30.50055|13.72186|   452|   27.8|       23.2|   4.6|      16.4|      19.454|      2.9| 0.507|20.

In [24]:



df_bristol = df_bristol.withColumnRenamed('Date Time', 'Date_Time')
#to impute the categorical values we used the aggregate function in which the most occuring value was used to impute the missing values

import pyspark.sql.functions as F
for col_name in ['DateStart', 'DateEnd','geo_point_2d', 'Date_Time']:
    common = df_bristol.dropna().groupBy(col_name).agg(F.count("*")).orderBy('count(1)', ascending=False).first()[col_name]
    df_bristol = df_bristol.withColumn(col_name, F.when(F.isnull(col_name), common).otherwise(df_bristol[col_name]))


df_bristol.show()

+--------------------+--------+--------+--------+------+-------+-----------+------+----------+------------+---------+------+--------+-----------+-----------+------+------------+--------------------+--------------------+--------------------+--------------------+-------+--------------------+
|           Date_Time|     NOx|     NO2|      NO|SiteID|   PM10|     NVPM10| VPM10|   NVPM2_5|       PM2_5|   VPM2_5|    CO|      O3|        SO2|Temperature|    RH|Air_Pressure|            Location|        geo_point_2d|           DateStart|             DateEnd|Current|     Instrument_Type|
+--------------------+--------+--------+--------+------+-------+-----------+------+----------+------------+---------+------+--------+-----------+-----------+------+------------+--------------------+--------------------+--------------------+--------------------+-------+--------------------+
|2013-08-23T07:00:...|51.54044|30.50055|13.72186|   452|   27.8|       23.2|   4.6|      16.4|      19.454|      2.9| 0.507|20.

In [42]:
df_bristol.coalesce(1).write.csv("data_cleaned1.csv")

In [29]:
Dict_Null = {col:df_bristol.filter(df_bristol[col].isNull()).count() for col in df_bristol.columns}
Dict_Null

{'Date_Time': 0,
 'NOx': 0,
 'NO2': 0,
 'NO': 0,
 'SiteID': 2,
 'PM10': 0,
 'NVPM10': 0,
 'VPM10': 0,
 'NVPM2_5': 0,
 'PM2_5': 0,
 'VPM2_5': 0,
 'CO': 0,
 'O3': 0,
 'SO2': 0,
 'Temperature': 0,
 'RH': 0,
 'Air_Pressure': 0,
 'Location': 0,
 'geo_point_2d': 0,
 'DateStart': 0,
 'DateEnd': 0,
 'Current': 0,
 'Instrument_Type': 0}

In [30]:
from pyspark.sql.functions import col, max as max_, min as min_


#Question 1 

df_bristol.agg({'CO': 'avg'}).collect()



#Question2 

#return the maximum value of CO, NOx for each station
df_bristol.groupBy('Location').agg(max_('CO'),max_('NOx')).collect()





[Row(Location='AURN Bristol Centre', max(CO)='9.5', max(NOx)='999.0'),
 Row(Location='AURN St Pauls', max(CO)='6.3', max(NOx)='999.23568'),
 Row(Location='Bath Road', max(CO)=' 0.507', max(NOx)='999.25'),
 Row(Location='Brislington Depot', max(CO)=' 0.507', max(NOx)='99.75'),
 Row(Location='CREATE Centre Roof', max(CO)=' 0.507', max(NOx)='116.892'),
 Row(Location='Cheltenham Road \\ Station Road', max(CO)=' 0.507', max(NOx)='99.99'),
 Row(Location='Colston Avenue', max(CO)=' 0.507', max(NOx)='990.0'),
 Row(Location='Fishponds Road', max(CO)=' 0.507', max(NOx)='995.25'),
 Row(Location='IKEA M32', max(CO)=' 0.507', max(NOx)='99.88'),
 Row(Location='Newfoundland Road Police Station', max(CO)=' 0.507', max(NOx)='998.75'),
 Row(Location='Old Market', max(CO)='4.2', max(NOx)='997.0'),
 Row(Location='Parson Street School', max(CO)=' 0.507', max(NOx)='998.0'),
 Row(Location='Rupert Street', max(CO)=' 0.507', max(NOx)='999.75'),
 Row(Location="Shiner's Garage", max(CO)=' 0.507', max(NOx)='99.75

In [31]:
df_bristol.withColumn("Air_Pressure",df_bristol.Air_Pressure.cast('double'))

DataFrame[Date_Time: string, NOx: string, NO2: string, NO: string, SiteID: string, PM10: string, NVPM10: string, VPM10: string, NVPM2_5: string, PM2_5: string, VPM2_5: string, CO: string, O3: string, SO2: string, Temperature: string, RH: string, Air_Pressure: double, Location: string, geo_point_2d: string, DateStart: string, DateEnd: string, Current: string, Instrument_Type: string]

In [32]:

#Question3 (selected the siteid and date after which applied the filter of 2016 and since the air pressure was the same throughput the min value was -999)
df_bristol.select('SiteID','Date_Time','Air_Pressure').filter(df_bristol.Date_Time.like('2016%')).collect()

[Row(SiteID='270', Date_Time='2016-05-02T04:00:00+00:00', Air_Pressure='-999'),
 Row(SiteID='270', Date_Time='2016-05-02T05:00:00+00:00', Air_Pressure='-999'),
 Row(SiteID='452', Date_Time='2016-05-02T06:00:00+00:00', Air_Pressure='-999'),
 Row(SiteID='215', Date_Time='2016-05-02T10:00:00+00:00', Air_Pressure='-999'),
 Row(SiteID='270', Date_Time='2016-05-02T10:00:00+00:00', Air_Pressure='-999'),
 Row(SiteID='215', Date_Time='2016-05-02T12:00:00+00:00', Air_Pressure='-999'),
 Row(SiteID='270', Date_Time='2016-05-02T15:00:00+00:00', Air_Pressure='-999'),
 Row(SiteID='463', Date_Time='2016-05-02T20:00:00+00:00', Air_Pressure='-999'),
 Row(SiteID='203', Date_Time='2016-05-02T23:00:00+00:00', Air_Pressure='-999'),
 Row(SiteID='270', Date_Time='2016-05-03T01:00:00+00:00', Air_Pressure='-999'),
 Row(SiteID='452', Date_Time='2016-05-03T03:00:00+00:00', Air_Pressure='-999'),
 Row(SiteID='452', Date_Time='2016-05-03T04:00:00+00:00', Air_Pressure='-999'),
 Row(SiteID='270', Date_Time='2016-05-03

In [33]:
#question 4 over here i first filtered out the colcumns with date 2019 and which had the Brislington depot as the location after this i took the average of the temperature of the filtered output

df_bristol.filter(df_bristol.Date_Time.like('2019%')).filter(df_bristol.Location.like('Brislington Depot')).agg({'Temperature': 'avg'}).show()   



+-----------------+
| avg(Temperature)|
+-----------------+
|9.513768899999402|
+-----------------+



In [34]:
#Question5 this was grouped use the date time and from there the max of the temeprature was recorded

df_bristol.groupby('Date_Time').agg(max_('Temperature')).collect()


[Row(Date_Time='1993-01-01T00:00:00+00:00', max(Temperature)='9.5137689'),
 Row(Date_Time='1993-01-01T01:00:00+00:00', max(Temperature)='9.5137689'),
 Row(Date_Time='1993-01-01T02:00:00+00:00', max(Temperature)='9.5137689'),
 Row(Date_Time='1993-01-01T03:00:00+00:00', max(Temperature)='9.5137689'),
 Row(Date_Time='1993-01-01T04:00:00+00:00', max(Temperature)='9.5137689'),
 Row(Date_Time='1993-01-01T05:00:00+00:00', max(Temperature)='9.5137689'),
 Row(Date_Time='1993-01-01T06:00:00+00:00', max(Temperature)='9.5137689'),
 Row(Date_Time='1993-01-01T07:00:00+00:00', max(Temperature)='9.5137689'),
 Row(Date_Time='1993-01-01T08:00:00+00:00', max(Temperature)='9.5137689'),
 Row(Date_Time='1993-01-01T09:00:00+00:00', max(Temperature)='9.5137689'),
 Row(Date_Time='1993-01-01T10:00:00+00:00', max(Temperature)='9.5137689'),
 Row(Date_Time='1993-01-01T11:00:00+00:00', max(Temperature)='9.5137689'),
 Row(Date_Time='1993-01-01T12:00:00+00:00', max(Temperature)='9.5137689'),
 Row(Date_Time='1993-01-0

In [35]:
#question 6 calculate O3 with respect to each location

df_bristol.groupBy('Location').agg(max_('O3')).collect()

[Row(Location='AURN Bristol Centre', max(O3)='98.0'),
 Row(Location='AURN St Pauls', max(O3)='99.98457'),
 Row(Location='Bath Road', max(O3)='40.65212'),
 Row(Location='Brislington Depot', max(O3)='40.65212'),
 Row(Location='CREATE Centre Roof', max(O3)='99.0'),
 Row(Location='Cheltenham Road \\ Station Road', max(O3)='40.65212'),
 Row(Location='Colston Avenue', max(O3)='40.65212'),
 Row(Location='Fishponds Road', max(O3)='40.65212'),
 Row(Location='IKEA M32', max(O3)='40.65212'),
 Row(Location='Newfoundland Road Police Station', max(O3)='40.65212'),
 Row(Location='Old Market', max(O3)='40.65212'),
 Row(Location='Parson Street School', max(O3)='40.65212'),
 Row(Location='Rupert Street', max(O3)='40.65212'),
 Row(Location="Shiner's Garage", max(O3)='40.65212'),
 Row(Location='Temple Meads Station', max(O3)='40.65212'),
 Row(Location='Temple Way', max(O3)='40.65212'),
 Row(Location='Trailer Portway P&R', max(O3)='40.65212'),
 Row(Location='Wells Road', max(O3)='79.22929')]

In [36]:
#Question 7 what is the max geo point of  each location

df_bristol.groupby('Location').agg(max_('geo_point_2d')).collect()


[Row(Location='AURN Bristol Centre', max(geo_point_2d)='51.4572041156,-2.58564914143'),
 Row(Location='AURN St Pauls', max(geo_point_2d)='51.4628294172,-2.58454081635'),
 Row(Location='Bath Road', max(geo_point_2d)='51.4425372726,-2.57137536073'),
 Row(Location='Brislington Depot', max(geo_point_2d)='51.4417471802,-2.55995583224'),
 Row(Location='CREATE Centre Roof', max(geo_point_2d)='51.447213417,-2.62247405516'),
 Row(Location='Cheltenham Road \\ Station Road', max(geo_point_2d)='51.4689385901,-2.5927241667'),
 Row(Location='Colston Avenue', max(geo_point_2d)='51.4552693825,-2.59664882861'),
 Row(Location='Fishponds Road', max(geo_point_2d)='51.4780449714,-2.53523027459'),
 Row(Location='IKEA M32', max(geo_point_2d)='51.4752847609,-2.56207998299'),
 Row(Location='Newfoundland Road Police Station', max(geo_point_2d)='51.4606738207,-2.58225341824'),
 Row(Location='Old Market', max(geo_point_2d)='51.4560189999,-2.58348949026'),
 Row(Location='Parson Street School', max(geo_point_2d)='5